<h1>Data setup</h1>
<h2>Use our function to read the data file</h2>

In [136]:
def read_311_data(datafile):
    import pandas as pd
    import numpy as np
    
    #Add the fix_zip function
    def fix_zip(input_zip):
        try:
            input_zip = int(float(input_zip))
        except:
            try:
                input_zip = int(input_zip.split('-')[0])
            except:
                return np.NaN
        if input_zip < 10000 or input_zip > 19999:
            return np.NaN
        return str(input_zip)
    
    #Read the file
    df = pd.read_csv(datafile,index_col='Unique Key')
    
    #fix the zip
    df['Incident Zip'] = df['Incident Zip'].apply(fix_zip)
    
    #drop all rows that have any nans in them (note the easier syntax!)
    
    df = df.dropna(how='any')
    
    #get rid of unspecified boroughs
    df = df[df['Borough'] != 'Unspecified']
    
    #Convert times to datetime and create a processing time column
    
    import datetime
    df['Created Date'] = df['Created Date'].apply(lambda x:datetime.datetime.strptime(x,'%m/%d/%Y %I:%M:%S %p'))
    df['Closed Date'] = df['Closed Date'].apply(lambda x:datetime.datetime.strptime(x,'%m/%d/%Y %I:%M:%S %p'))
    df['processing_time'] =  df['Closed Date'] - df['Created Date']
    
    #Finally, get rid of negative processing times and return the final data frame
    
    df = df[df['processing_time']>=datetime.timedelta(0,0,0)]
    
    return df
    

In [137]:
datafile = "nyc_311_data_subset.csv"
data = read_311_data(datafile)

/Users/hardeepjohar/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


<h1>Plotting data on google maps</h1>
<h2>gmplot library</h2>
https://github.com/vgm64/gmplot

In [138]:
!pip install gmplot --upgrade

Requirement already up-to-date: gmplot in /Users/hardeepjohar/anaconda/lib/python3.5/site-packages
Requirement already up-to-date: requests in /Users/hardeepjohar/anaconda/lib/python3.5/site-packages (from gmplot)
Requirement already up-to-date: chardet<3.1.0,>=3.0.2 in /Users/hardeepjohar/anaconda/lib/python3.5/site-packages (from requests->gmplot)
Requirement already up-to-date: idna<2.6,>=2.5 in /Users/hardeepjohar/anaconda/lib/python3.5/site-packages (from requests->gmplot)
Requirement already up-to-date: certifi>=2017.4.17 in /Users/hardeepjohar/anaconda/lib/python3.5/site-packages (from requests->gmplot)
Requirement already up-to-date: urllib3<1.22,>=1.21.1 in /Users/hardeepjohar/anaconda/lib/python3.5/site-packages (from requests->gmplot)


<h3>Our data dataframe contains latitudes and longitudes for each complaint.</h3>
<h3>We can draw a heatmap that will help us see the relative concentration of complaints using lats and lons</h3>



<h3>Set up the map</h3>
<h4>GoogleMapPlotter constructor</h4>
<ul>
<li>GoogleMapPlotter(center_lat, center_lng, zoom)
<li>from_geocode(location_string,zoom)


In [139]:

import gmplot
#gmap = gmplot.GoogleMapPlotter(40.7128, 74.0059, 8)


gmap = gmplot.GoogleMapPlotter.from_geocode("New York",10)


<h3>Then generate the heatmap passing the two data series (latitude and longitude) to the function</h3>

In [140]:
#Then generate a heatmap using the latitudes and longitudes
gmap.heatmap(data['Latitude'], data['Longitude'])

<h3>Save the heatmap to an html file</h3>
<h4>The html file can be viewed, printed, or included in another html page</h4>

In [141]:
gmap.draw('incidents3.html')

<h1>Let's do some grouping operations</h1>

<h2>Incidents by Borough</h2>

In [ ]:
%matplotlib inline

<h2>Group data by borough and plot a bar chart of the incident count</h2>

In [ ]:

borough_group = data.groupby('Borough')
borough_group.size().plot(kind='bar')
#kind can be 'hist', 'scatter'

<h1>Incidents by Agency</h1>

In [ ]:
agency_group = data.groupby('Agency')
agency_group.size().plot(kind='bar')

<h2>Let's combine the two in a single graph</h2>


In [ ]:

agency_borough = data.groupby(['Agency','Borough'])
agency_borough.size().plot(kind='bar')


<h2>This is quite unreadable and pointless!</h2>

<h3>We can unstack the groups so that we get borough by agency</h3>

In [ ]:
agency_borough.size().unstack().plot(kind='bar')

<h3>Increase the size of the image and add a title</h3>

In [ ]:

agency_borough = data.groupby(['Agency','Borough'])
agency_borough.size().unstack().plot(kind='bar',title="Incidents in each Agency by Borough",figsize=(15,15))


<h1>Digression: The pandas groupby function</h1>

<h4>You can use functions to group data</h4>

In [ ]:
import pandas as pd
writers = pd.DataFrame({'Author':['George Orwell','John Steinbeck',
                                  'Pearl Buck','Agatha Christie'],
                        'Country':['UK','USA','USA','UK'],
                        'Gender':['M','M','F','F'],
                        'Age':[46,66,80,85]})
                        


In [ ]:
writers

<h4>Group by country</h4>

In [ ]:
grouped = writers.groupby('Country')
grouped.first()
#grouped.last()
#grouped.sum()
#grouped.mean()
#grouped.apply(sum)

In [ ]:
grouped.groups

<h4>Group by multiple columns</h4>

In [ ]:
grouped = writers.groupby(['Country','Gender'])
grouped.groups

<h4>Group by age groups</h4>

In [ ]:
def age_groups(df,index,col):
    print(index,col)
    if df[col].iloc[index] < 30:
        return 'Young'
    if df[col].iloc[index] < 60:
        return 'Middle'
    else:
        return 'Old'

In [ ]:
writers['Age'].iloc[0]

In [ ]:
grouped = writers.groupby(lambda x: age_groups(writers,x,'Age'))
grouped.groups

<h2>Grouping by the values in a column</h2>
<h3>For example, grouping the data by values in a column that are greater than or less than zero</h3>

In [ ]:
people = pd.DataFrame(np.random.randn(5, 5), columns=['a', 'b', 'c', 'd', 'e'], index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people

<h3>Write a function that takes three arguments - a dataframe, an index, and a column name and returns the grouping for that row</h3>

In [ ]:
def GroupColFunc(df, ind, col):
    if df[col].loc[ind] > 0:
        return 'Group1'
    else:
        return 'Group2'

In [ ]:
people.groupby(lambda x: GroupColFunc(people, x, 'a')).groups

<h2>Now we can compute stats on these groups</h2>

In [ ]:
print(people.groupby(lambda x: GroupColFunc(people, x, 'a')).mean())
print(people.groupby(lambda x: GroupColFunc(people, x, 'a')).std())

<h1>Incidents by time</h1>
<p>We know the creation date of each incident so we can build a bar graph of number of incidents by month
<p>Not particularly useful with a few months data but if we had all data from 2010, we could use this sort of
analysis to eyeball trends and seasonality
<p>We're going to need to do some data manipulation for this

<h3>We'll start by creating a new date field yyyymm

In [ ]:
import datetime
data['yyyymm'] = data['Created Date'].apply(lambda x:datetime.datetime.strftime(x,'%Y%m'))

In [ ]:
data['yyyymm']

In [ ]:
date_agency = data.groupby(['yyyymm','Agency'])
date_agency.size().unstack().plot(kind='bar',figsize=(15,15))

<h1>Examining agencies</h1>

<h2>We'll look at the frequency by agency and report the top 5 values</h2>

In [ ]:
data.groupby('Agency').size().sort_values(ascending=False)

In [ ]:
data.groupby('Agency').size().sort_values(ascending=False).plot(kind='bar', figsize=(20,4))

<h3>We can drill down into complaints by Agency by borough</h3>

In [ ]:
agency_borough = data.groupby(['Agency', 'Borough']).size().unstack()

In [ ]:
agency_borough

<h3>We can create 'top 5 Agency' subplots subplots for each borough</h3>

In [ ]:
#We'll arrange the subplots in two rows and three columns. 
#Since we have only 5 boroughs, one plot will be blank
COL_NUM = 2
ROW_NUM = 3
import matplotlib.pyplot as plt
fig, axes = plt.subplots(ROW_NUM, COL_NUM, figsize=(12,12))

for i, (label, col) in enumerate(agency_borough.iteritems()): 
    ax = axes[int(i/COL_NUM), i%COL_NUM]
    col = col.sort_values(ascending=False)[:5] 
    col.plot(kind='barh', ax=ax)
    ax.set_title(label)

plt.tight_layout() 

In [ ]:
for i, (label, col) in enumerate(agency_borough.iteritems()): 
    print(i,label,col)

<h1>Processing time</h1>
<h2>We can compute simple statistics on processing time</h2>

In [ ]:
grouped = data[['processing_time','Borough']].groupby('Borough')

In [ ]:
grouped.describe()

<h3>But it is easier to convert the timedelta processing_time into floats for calculation purposes</h3>

In [ ]:
import numpy as np
#The time it takes to process. Cleaned up
data['float_time'] =data['processing_time'].apply(lambda x:x/np.timedelta64(1, 'D'))

In [ ]:
data

<h2>Now we can compute stats easily</h2>

In [ ]:
grouped = data[['float_time','Agency']].groupby('Agency')
grouped.mean().sort_values('float_time',ascending=False)

In [ ]:
data['float_time'].hist(bins=50)

<h1>Other useful visualization libraries</h1>
<h2>seaborn: https://seaborn.pydata.org/</h2>
<h2>bokeh: http://bokeh.pydata.org/en/latest/</h2>
<h2>plotly: https://plot.ly/</h2>
